<a href="https://colab.research.google.com/github/pjot726/PyTorchTutorials/blob/main/Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

Here, we import needed packages from PyTorch. torchvision.datasets contains the datasets of interest, and DataLoader wraps an iterable around the dataset.

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

We now download the training and test data from the open datasets.

In [4]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



We now pass this dataset to the DataLoader object, which wraps an iterable over the dataset. By specifying a batch size of 64, the element of the iterable will all be 64 features and labels.

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Now, we create our neural network.

In [6]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


We now begin training the model, and to do so we define a loss function and an optimizer.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

We now define the training function, where we make predictions on the training set and backpropagates the prediction error to adjust model parameters.

In [8]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

We can also write a function to test the model's performance against the test set.

In [9]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We now train the model.

In [10]:
epochs = 5
for t in range(epochs):
  print(f"Epoch: {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch: 1
-------------------------------
loss: 2.295105 [   64/60000]
loss: 2.284853 [ 6464/60000]
loss: 2.263896 [12864/60000]
loss: 2.262896 [19264/60000]
loss: 2.252256 [25664/60000]
loss: 2.212262 [32064/60000]
loss: 2.228522 [38464/60000]
loss: 2.190374 [44864/60000]
loss: 2.182106 [51264/60000]
loss: 2.159206 [57664/60000]
Test Error: 
 Accuracy: 45.9%, Avg loss: 2.148978 

Epoch: 2
-------------------------------
loss: 2.158418 [   64/60000]
loss: 2.148803 [ 6464/60000]
loss: 2.084264 [12864/60000]
loss: 2.102384 [19264/60000]
loss: 2.061581 [25664/60000]
loss: 1.988152 [32064/60000]
loss: 2.029553 [38464/60000]
loss: 1.940533 [44864/60000]
loss: 1.942413 [51264/60000]
loss: 1.882550 [57664/60000]
Test Error: 
 Accuracy: 55.3%, Avg loss: 1.871028 

Epoch: 3
-------------------------------
loss: 1.909532 [   64/60000]
loss: 1.878474 [ 6464/60000]
loss: 1.748956 [12864/60000]
loss: 1.791502 [19264/60000]
loss: 1.696864 [25664/60000]
loss: 1.636287 [32064/60000]
loss: 1.673979 [384

We now save and load the model both to and from a file, to show how this can be done.

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
loadedModel = NeuralNetwork()
loadedModel.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

We now can use this model to make predictions.

In [14]:
classes= [ 
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: {predicted}, Actual: {actual}')

Predicted: Pullover, Actual: Pullover
